# Building LLM-based applications for writting discharge summary

In this notebook, we will build an application to writting discharge summary.

For required packages, please run:

```bash
poetry install
```

There are a few methods explored in this notebook:
1. Stuff with human and system prompt
2. Stuff with human prompt
3. Decompose method
4. MapReduce method
5. Refine method

In [ ]:
import os
import sys
from pathlib import Path

import nest_asyncio
from dotenv import load_dotenv
from IPython.display import Markdown

nest_asyncio.apply()

load_dotenv()

# Setup import paths
project_folder = Path(os.getcwd()).parent
sys.path.append(str(project_folder))

# Import local modules
from helpers import count_tokens, init_model

# Setup

In [ ]:
# Set up the GPT model
llm, _ = init_model("gpt-35", temperature=0.0)

# Set the data paths
output_folder = project_folder / "data" / "output"
case_dir = project_folder / "data" / 'processed'
if not case_dir.exists():
    raise FileNotFoundError(f"Cases directory not found at {case_dir}")
if not output_folder.exists():
    output_folder.mkdir(parents=True)

# Read cases

In [ ]:
from helpers import read_all_cases

case_dict = read_all_cases(base_dir=case_dir,
                           filter_specialty='Orthopaedics',
                           filter_language='original')
# Pick first case from the case_dict
explore_case = list(case_dict.values())[0]

In [ ]:
# Check the case text and tokens, note that some models have limits on the number of tokens per minute
count_tokens(explore_case.text, model_name=llm.model_name)

# Basic method (Naive)

In [ ]:
from basic.basic import basic_chain

out = basic_chain(llm=llm).dual.invoke({"notes": explore_case.text})
Markdown(out)

# Decompose Technique

In [ ]:
from decompose import single_decompose

out = single_decompose(case=explore_case.object, llm=llm)
Markdown(out)

# Map Reduce Prompt Techniques

In [ ]:
from map_reduce import create_mapreduce_df

mapreduce_sum = create_mapreduce_df(case=explore_case.object, llm=llm, n=2)
mapreduce_sum.to_csv(output_folder / "mapreduce_final.csv")

# Refine Prompt Techniques

In [ ]:
from refine import create_refine_df

refine_sum = create_refine_df(case=explore_case.object, llm=llm, n=2)
refine_sum.to_csv(output_folder / "refine_final.csv")